# ASSIGNMENT PART START FROM HERE

# *Using all Sensor Data*

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/bbh

/content/drive/MyDrive/bbh


In [3]:
import os
os.listdir('.')

['training', 'testing']

In [4]:
%cd /content/drive/MyDrive/bbh
import os
os.listdir('.')


/content/drive/MyDrive/bbh


['training', 'testing']

In [5]:
import numpy as np

train_accelerometer_path = './training/trainAccelerometer.npy'
train_gravity_path =  './training/trainGravity.npy'
train_gyroscope_path =  './training/trainGyroscope.npy'
train_jins_accelerometer_path =  './training/trainJinsAccelerometer.npy'
train_jins_gyroscope_path =  './training/trainJinsGyroscope.npy'
train_label_path = './training/trainLabels.npy'
train_linear_acceleration_path  = './training/trainLinearAcceleration.npy'
train_ms_accelerometer_path = './training/trainMSAccelerometer.npy'
train_ms_gyroscope_path = './training/trainMSGyroscope.npy'
train_magnetometer_path = './training/trainMagnetometer.npy'

train_accelerometer = np.load(train_accelerometer_path)
train_gravity =  np.load(train_gravity_path)
train_gyroscope = np.load(train_jins_accelerometer_path)
train_jins_accelerometer =  np.load(train_jins_accelerometer_path)
train_jins_gyroscope =  np.load(train_jins_gyroscope_path )
train_label = np.load(train_label_path)
train_linear_acceleration  = np.load(train_linear_acceleration_path)
train_ms_accelerometer = np.load(train_ms_accelerometer_path)
train_ms_gyroscope = np.load(train_ms_gyroscope_path)
train_magnetometer = np.load(train_magnetometer_path)


test_accelerometer_path = './testing/testAccelerometer.npy'
test_gravity_path =  './testing/testGravity.npy'
test_gyroscope_path =  './testing/testGyroscope.npy'
test_jins_accelerometer_path =  './testing/testJinsAccelerometer.npy'
test_jins_gyroscope_path =  './testing/testJinsGyroscope.npy'
test_label_path = './testing/testLabels.npy'
test_linear_acceleration_path  = './testing/testLinearAcceleration.npy'
test_ms_accelerometer_path = './testing/testMSAccelerometer.npy'
test_ms_gyroscope_path = './testing/testMSGyroscope.npy'
test_magnetometer_path = './testing/testMagnetometer.npy'

test_accelerometer = np.load(test_accelerometer_path)
test_gravity =  np.load(test_gravity_path)
test_gyroscope = np.load(test_jins_accelerometer_path)
test_jins_accelerometer =  np.load(test_jins_accelerometer_path)
test_jins_gyroscope =  np.load(test_jins_gyroscope_path )
test_label = np.load(test_label_path)
test_linear_acceleration  = np.load(test_linear_acceleration_path)
test_ms_accelerometer = np.load(test_ms_accelerometer_path)
test_ms_gyroscope = np.load(test_ms_gyroscope_path)
test_magnetometer = np.load(test_magnetometer_path)



print(train_label.shape)
print(test_label.shape)


(2284,)
(2288,)


In [7]:
#sensors to use
sensors = [train_accelerometer, train_gravity, train_gyroscope, train_jins_accelerometer,
           train_jins_gyroscope, train_linear_acceleration,train_ms_accelerometer,
           train_ms_gyroscope, train_magnetometer]



In [8]:
from scipy.fft import fft
import numpy as np

def calculate_spectral_energy(column):
    # Compute the magnitude spectrum using the Fast Fourier Transform
    spectrum = np.abs(np.fft.fft(column))

    # Calculate the spectral energy as the sum of squared magnitudes
    energy = np.sum(np.square(spectrum))

    return energy


def first_order_meen(column):
    # Calculate the differences between consecutive elements
    differences = np.diff(column)

    # Calculate the mean of the differences
    mean = np.mean(differences)

    return mean

def interquartile_range(data):
    # Calculate the 25th percentile (first quartile)
    q1 = np.percentile(data, 25)

    # Calculate the 75th percentile (third quartile)
    q3 = np.percentile(data, 75)

    # Calculate the interquartile range
    iqr = q3 - q1

    return iqr

def zero_crossing(data):
    # Find the indices where the sign of consecutive elements change
    sign_changes = np.where(np.diff(np.sign(data)))[0]

    return len(sign_changes)



def norm_first_order_meen(column):
    # Ensure column is a one-dimensional array
    column = np.array(column).ravel()

    # Calculate the differences between consecutive elements
    differences = np.diff(column)

    # Calculate the mean of the differences
    mean = np.mean(differences)

    # Normalize the mean by dividing by the standard deviation
    std = np.std(differences)
    if std != 0 and not np.isnan(std):
      norm_mean = mean / std
    else:
      norm_mean = np.mean(column)


    return norm_mean


def second_order_meen(column):
    # Calculate the differences between consecutive elements
    differences = np.diff(column)

    # Calculate the mean of the squared differences
    squared_diff_mean = np.mean(np.square(differences))

    return squared_diff_mean


def norm_second_order_meen(column):
    # Calculate the differences between consecutive elements
    differences = np.diff(column)

    # Calculate the mean of the squared differences
    squared_diff_mean = np.mean(np.square(differences))

    # Normalize the mean by dividing by the standard deviation
    std = np.std(differences)

    if std != 0 and not np.isnan(std):
      norm_mean = squared_diff_mean / std
    else:
      norm_mean = np.mean(column)


    return norm_mean


def spectral_entrpy(column):
    # Compute the magnitude spectrum using the Fast Fourier Transform
    spectrum = np.abs(np.fft.fft(column))

    # Normalize the spectrum
    sum_spectrum = np.sum(spectrum)
    if sum_spectrum != 0 and not np.isnan(sum_spectrum):
      normalized_spectrum = spectrum / sum_spectrum
    else:
      # Handle the scenario when sum_spectrum is zero or invalid
      normalized_spectrum = np.zeros_like(spectrum)  # or any other appropriate value

    # Compute the entropy
    zero_mask = normalized_spectrum == 0
    normalized_spectrum[zero_mask] = np.finfo(float).eps  # Replace zeros with a small epsilon value

    entropy = -np.sum(normalized_spectrum * np.log2(normalized_spectrum))


    return entropy


def kurtosis_(column):
    # Calculate the kurtosis using the fourth central moment
    mean = np.mean(column)
    centered_values = column - mean
    squared_values = centered_values ** 2
    fourth_moment = np.mean(squared_values ** 2)
    variance = np.var(column)

    if variance != 0 and not np.isnan(variance):
      kurt = fourth_moment / (variance ** 2)
    else:
      # Handle the scenario when variance is zero or invalid
      kurt = np.mean(column)  # or any other appropriate value

    return kurt

def skewness_(column):
    # Calculate the skewness using the third central moment
    mean = np.mean(column)
    centered_values = column - mean
    cubed_values = centered_values ** 3
    third_moment = np.mean(cubed_values)
    variance = np.var(column)

    if variance != 0 and not np.isnan(variance):
      skew = third_moment / (variance ** (3/2))
    else:
      skew = np.mean(column)  # or any other appropriate value


    return skew

def auto_correlation_(column):
    # Calculate the auto-correlation using the cross-correlation function
    auto_corr = np.correlate(column, column, mode='full')
    auto_corr = auto_corr[len(auto_corr)//2]

    return auto_corr




In [9]:
feature_matrixes = []
for sensor in sensors:
  sensor_feature_matrix = np.zeros((sensor.shape[0], 54))
  feature_matrixes.append(sensor_feature_matrix)
  for i in range(sensor.shape[0]):
    result_matrix = np.zeros((18,3))
    example = sensor[i]

    for j in range(3):
      col = example[:, j]
      maximum = np.max(col)
      minimum = np.min(col)
      avg = np.average(col)
      stand_deviation = np.std(col)
      zero_cross = zero_crossing(col)
      percentile_20 = np.percentile(col, 20)
      percentile_50 = np.percentile(col, 50)
      percentile_80 = np.percentile(col, 80)
      interquartile = interquartile_range(col)
      skewness = skewness_(col)
      kurtosis = kurtosis_(col)
      auto_corr = auto_correlation_(col)
      first_order_mean_val = first_order_meen(col)
      norm_first_order_mean_val = norm_first_order_meen(col)
      second_order_mean_val = second_order_meen(col)
      norm_second_order_mean_val = norm_second_order_meen(col)
      spectral_energy = calculate_spectral_energy(col)
      spectral_entropy = spectral_entrpy(col)
      values = [maximum,minimum, avg, stand_deviation, zero_cross, percentile_20, percentile_50, percentile_80
                ,  interquartile ,skewness,kurtosis, auto_corr, first_order_mean_val, norm_first_order_mean_val,
                second_order_mean_val, norm_second_order_mean_val ,
                spectral_energy , spectral_entropy ]

      result_matrix[:, j] = values
    reshaped_matrix = result_matrix.flatten()
    sensor_feature_matrix[i] = reshaped_matrix


  # all_sensor_feature_Matrix = np.concatenate((all_sensor_feature_Matrix , sensor_feature_matrix), axis=1)





In [10]:
all_sensors_feature_matrix = np.concatenate((feature_matrixes[0], feature_matrixes[1], feature_matrixes[2], feature_matrixes[3], feature_matrixes[4], feature_matrixes[5]
                , feature_matrixes[6], feature_matrixes[7], feature_matrixes[8]), axis=1)

for i in feature_matrixes:
  print(i.shape)

print("all", all_sensors_feature_matrix.shape)
print('train_label', train_label.shape)

(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
(2284, 54)
all (2284, 486)
train_label (2284,)


## Checking NaN in features

In [11]:
import math
index = -1
count = 0
for f_matrix in feature_matrixes:
  index += 1
  for i in range(2284):
    for j in range(51):

      if math.isnan(f_matrix[i][j]):
        count += 1
        # print("Feature Matrix: " ,  index, " with value ", feature_matrix[i][j])

print("count", count)
# print(feature_matrixes[0])

count 0


## **TRAINING THE MODEL**

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# algo=RandomForestClassifier(n_estimators=100 , criterion='gini')
# # n_estimators tell us how many number of trees we want to generate

# # Train the model
# algo.fit( #feature_matrix is taken as a input and test_labels are the labels



# Create an imputer to handle missing values
imputer = SimpleImputer(strategy='mean')

# Create a pipeline with the imputer and the classifier
pipeline = make_pipeline(imputer, RandomForestClassifier(n_estimators=100 , criterion='gini'))

# Fit the pipeline to your data
pipeline.fit(all_sensors_feature_matrix , train_label)


Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('randomforestclassifier', RandomForestClassifier())])

In [14]:
from sklearn.ensemble import HistGradientBoostingClassifier

# Create an instance of the classifier
clf = HistGradientBoostingClassifier()

# Fit the classifier to your data
clf.fit(all_sensors_feature_matrix, train_label)

HistGradientBoostingClassifier()

## **TESTING PHASE**

# Feature Extraction from test

In [15]:
test_sensors = [test_accelerometer, test_gravity, test_gyroscope, test_jins_accelerometer,
           test_jins_gyroscope, test_linear_acceleration,test_ms_accelerometer,
           test_ms_gyroscope, test_magnetometer]

In [ ]:
test_feature_matrixes = []
for test_sensor in test_sensors:
  test_sensor_feature_matrix = np.zeros((test_sensor.shape[0], 54))
  test_feature_matrixes.append(test_sensor_feature_matrix)
  for i in range(sensor.shape[0]):
    result_matrix = np.zeros((18,3))
    example = test_sensor[i]

    for j in range(3):
      # if example.size == 0 or example.shape == () :
      #   print(sensor)
      #   print("row: ", i)
      #   break
      col = example[:, j]
      maximum = np.max(col)
      minimum = np.min(col)
      avg = np.average(col)
      stand_deviation = np.std(col)
      zero_cross = zero_crossing(col)
      percentile_20 = np.percentile(col, 20)
      percentile_50 = np.percentile(col, 50)
      percentile_80 = np.percentile(col, 80)
      interquartile = interquartile_range(col)
      skewness = skewness_(col)
      kurtosis = kurtosis_(col)
      auto_corr = auto_correlation_(col)
      first_order_mean_val = first_order_meen(col)
      norm_first_order_mean_val = norm_first_order_meen(col)
      second_order_mean_val = second_order_meen(col)
      norm_second_order_mean_val = norm_second_order_meen(col)
      spectral_energy = calculate_spectral_energy(col)
      spectral_entropy = spectral_entrpy(col)
      values = [maximum,minimum, avg, stand_deviation, zero_cross, percentile_20, percentile_50, percentile_80
                ,  interquartile ,skewness,kurtosis, auto_corr, first_order_mean_val, norm_first_order_mean_val,
                second_order_mean_val, norm_second_order_mean_val ,
                spectral_energy , spectral_entropy ]

      result_matrix[:, j] = values
    reshaped_matrix = result_matrix.flatten()
    test_sensor_feature_matrix[i] = reshaped_matrix


  # all_sensor_feature_Matrix = np.concatenate((all_sensor_feature_Matrix , sensor_feature_matrix), axis=1)





In [ ]:
test_all_sensors_feature_matrix = np.concatenate((test_feature_matrixes[0], test_feature_matrixes[1], test_feature_matrixes[2],
                                                  test_feature_matrixes[3], test_feature_matrixes[4], test_feature_matrixes[5]
                , test_feature_matrixes[6], test_feature_matrixes[7], test_feature_matrixes[8]), axis=1)

for i in test_feature_matrixes:
  print(i.shape)

print("all", test_all_sensors_feature_matrix.shape)
print('train_label', test_label.shape)

(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
(2288, 54)
all (2288, 486)
train_label (2288,)


In [ ]:
# In[59]:


pipeline.predict(test_all_sensors_feature_matrix)










array([17, 13, 49, ..., 49, 49, 49])

In [ ]:
clf.predict(test_all_sensors_feature_matrix)

array([17, 10,  5, ..., 49, 49, 49])

In [ ]:
# score for random forest
pipeline.score(test_all_sensors_feature_matrix,test_label)

0.6988636363636364

In [ ]:
#score for HistGradientBoostingClassifier
clf.score(test_all_sensors_feature_matrix,test_label)

0.6437937062937062

# For each Sensor

In [ ]:
random_forests_per_sensor = []
scores_per_sensor = []
for i in range(len(sensors)):
  random_forests_per_sensor.append(RandomForestClassifier(n_estimators=100 , criterion='gini') )

  # Train the model
  random_forests_per_sensor[i].fit(feature_matrixes[i], train_label)
  random_forests_per_sensor[i].predict(test_feature_matrixes[i])
  scores_per_sensor.append( random_forests_per_sensor[i].score(test_feature_matrixes[i], test_label) )


print(scores_per_sensor)


[0.36363636363636365, 0.3505244755244755, 0.40166083916083917, 0.40821678321678323, 0.4143356643356643, 0.2928321678321678, 0.5362762237762237, 0.40777972027972026, 0.3553321678321678]


# Using Best Score Sensors

In [ ]:
# best_score_feature_matrix_concat = np.concatenate((feature_matrixes[0], feature_matrixes[2], feature_matrixes[3], feature_matrixes[4], feature_matrixes[6], feature_matrixes[7], feature_matrixes[8]), axis=1)
# test_best_feature_matrix_concat =  np.concatenate((test_feature_matrixes[0], test_feature_matrixes[2], test_feature_matrixes[3], test_feature_matrixes[4], test_feature_matrixes[6], test_feature_matrixes[7], test_feature_matrixes[8]), axis=1)
# print(best_score_feature_matrix_concat.shape)
best_score_feature_matrix_concat = np.concatenate((feature_matrixes[2], feature_matrixes[3], feature_matrixes[4], feature_matrixes[6], feature_matrixes[7], feature_matrixes[8]), axis=1)
test_best_feature_matrix_concat =  np.concatenate((test_feature_matrixes[0], test_feature_matrixes[2], test_feature_matrixes[3], test_feature_matrixes[4], test_feature_matrixes[6], test_feature_matrixes[7], test_feature_matrixes[8]), axis=1)
print(best_score_feature_matrix_concat.shape)

(2284, 378)


# Traing and testing and Score

In [ ]:
best_model = RandomForestClassifier(n_estimators=100 , criterion='gini')
best_model.fit(best_score_feature_matrix_concat, train_label)
best_model.predict(test_best_feature_matrix_concat)
best_model.score(test_best_feature_matrix_concat, test_label)


0.7102272727272727